In [ ]:
# Use Python 3.12 enviroment
%pip install pillow mutagen librosa soundfile numpy

In [3]:
import base64
import shutil
from io import BytesIO
from pathlib import Path
from PIL import Image
from mutagen.oggvorbis import OggVorbis
from mutagen.flac import Picture
import librosa


script_dir = Path(__file__).parent
input_dir = script_dir
output_dir = script_dir.parent / "Assets" / "StreamingAssets" / "Music"
output_dir.mkdir(parents=True, exist_ok=True)


def make_square(img):
    w, h = img.size
    if w == h:
        return img
    side = min(w, h)
    left = (w - side) // 2
    top = (h - side) // 2
    return img.crop((left, top, left + side, top + side))


def calculate_bpm(path):
    try:
        y, sr = librosa.load(path, sr=None, mono=True)
        tempo, _ = librosa.beat.beat_track(y=y, sr=sr)
        return int(round(float(tempo)))
    except Exception as e:
        print("Error calculating BPM:", e)
        return None


def process_file(src_path):
    dst_path = output_dir / src_path.name
    shutil.copy2(src_path, dst_path)

    audio = OggVorbis(dst_path)

    # ---------------------------------------------------
    # 1. CALCULAR BPM Y GUARDAR COMO METADATA
    # ---------------------------------------------------
    bpm = calculate_bpm(dst_path)
    if bpm is not None:
        audio["BPM"] = str(bpm)

    # ---------------------------------------------------
    # 2. PROCESAR Y RE-EMBEDIR PORTADA CUADRADA
    # ---------------------------------------------------
    if "metadata_block_picture" in audio:
        pic_b64 = audio["metadata_block_picture"][0]
        raw = base64.b64decode(pic_b64)

        pic = Picture(raw)
        img = Image.open(BytesIO(pic.data)).convert("RGB")
        img_sq = make_square(img)

        buf = BytesIO()
        img_sq.save(buf, format="JPEG", quality=97)
        jpg_bytes = buf.getvalue()

        newpic = Picture()
        newpic.type = 3
        newpic.mime = "image/jpeg"
        newpic.desc = ""
        newpic.width, newpic.height = img_sq.size
        newpic.depth = 24
        newpic.data = jpg_bytes

        encoded = base64.b64encode(newpic.write()).decode("ascii")
        audio["metadata_block_picture"] = [encoded]

    # Guardar cambios
    audio.save()


def main():
    for f in input_dir.iterdir():
        if f.is_file() and f.suffix.lower() == ".ogg":
            process_file(f)

    for f in input_dir.iterdir():
        if f.is_file() and f.suffix.lower() == ".ogg":
            f.unlink()

    print("Procesamiento finalizado. Archivos en:", output_dir)


if __name__ == "__main__":
    main()

NameError: name '__file__' is not defined